In [6]:
import os
from typing import Optional
from openai import OpenAI
from dotenv import load_dotenv
from abc import ABC, abstractmethod
load_dotenv()

client = OpenAI()

client = OpenAI()

class Agent(ABC):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        
        model = model if model else os.getenv("OPENAI_DEFAULT_MODEL")
        
        self.memory = []
        self.model = model
        
        if system_prompt:
            self.memory.append({"role": "system", "content": system_prompt})
    
    def add_to_memory(self, role, message):
        self.memory.append({"role": role, "content": message})
    
    @abstractmethod
    def run(self,prompt):
        """User must define this method. Run the agent"""
    
class ConversationAgent(Agent):
    
    def __init__(self, 
        model: Optional[str] = None, 
        system_prompt: Optional[str] = None
    ) -> None:
        super().__init__(model,system_prompt)
    
    def run(self,prompt):

        self.add_to_memory("user", prompt)
        
        chat_completion = client.chat.completions.create(
            messages=self.memory,
            model=self.model,
        )
        response=chat_completion.choices[0].message.content
        
        self.add_to_memory("assistant", response)
        
        return response
    

In [7]:
#Define the agents
Emotion = ConversationAgent(model="gpt-3.5-turbo", system_prompt="You are a emocional expert. Response to the user in short to help them feel understood.")
Facts = ConversationAgent(model="gpt-3.5-turbo", system_prompt="You are a facts expert. Response to the user in short to create a plan to help them with facts.")
Helper = ConversationAgent(model="gpt-4", system_prompt="You receive information to respond to the user in short and give them a plan to help them with facts and emotions.")

# Define una solicitud de usuario más detallada para una tarea específica.
user_request = """
Y si lo hacemos, si nos unimos y sacamos a este país de la profundidad de esta 
crisis; si hacemos que nuestra gente vuelva a trabajar y volvemos a poner en 
marcha el motor de nuestra prosperidad; si enfrentamos los desafíos de nuestros 
tiempos y hacemos un llamado a ese espíritu perdurable de un estadounidense 
que no se da por vencido, entonces algún día, dentro de muchos años, nuestros 
hijos podrán decirles a sus hijos que este fue el momento en que hicimos, en 
palabras que están talladas en este recinto, «algo digno de ser recordado». 
Gracias, que Dios los bendiga y que Dios bendiga a Estados 
Unidos de América”

Evalua este discurso presidencial
"""

# El UserAgent recoge la solicitud y pide clarificaciones si son necesarias.
emotion_response = Emotion.run(user_request)
facts_response = Facts.run(user_request)
helper_response = Helper.run(emotion_response + facts_response)

# Imprime la interacción completa
print(f"---> User:\n\n{user_request}\n")
print(f"---> emotion_response:\n\n{emotion_response}\n")
print(f"---> facts_response:\n\n{facts_response}\n")
print(f"---> helper_response:\n\n{helper_response}\n")



---> User:


Y si lo hacemos, si nos unimos y sacamos a este país de la profundidad de esta 
crisis; si hacemos que nuestra gente vuelva a trabajar y volvemos a poner en 
marcha el motor de nuestra prosperidad; si enfrentamos los desafíos de nuestros 
tiempos y hacemos un llamado a ese espíritu perdurable de un estadounidense 
que no se da por vencido, entonces algún día, dentro de muchos años, nuestros 
hijos podrán decirles a sus hijos que este fue el momento en que hicimos, en 
palabras que están talladas en este recinto, «algo digno de ser recordado». 
Gracias, que Dios los bendiga y que Dios bendiga a Estados 
Unidos de América”

Evalua este discurso presidencial


---> emotion_response:

Este discurso presidencial muestra una gran voluntad de unir al país y superar la crisis actual. Expresa el deseo de trabajar juntos para reconstruir la economía y recuperar la prosperidad. También destaca la importancia de enfrentar los desafíos actuales y apelar al espíritu resiliente de los es